In [1]:
import cv2 as cv
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from sklearn.tree import DecisionTreeClassifier 
from sklearn.utils import shuffle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#import mahotas
import h5py
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

/home/franklinsierra/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# <span style="color:red">============== DECISION TREE ===============</span>

# <span style="color:red"> DT para imagenes promedio</span>

In [3]:
path_classes = "/home/franklinsierra/Documentos/Biomedica/Polipos/notebooks/mix_avg_img/"
classes_names = os.listdir(path_classes)
print (classes_names)
dic_img ={}

for i, name_class in enumerate(classes_names):
    dic_img[classes_names[i]] = np.array(os.listdir(path_classes+classes_names[i]))
    print ("class: ", name_class, "total ", len(dic_img[classes_names[i]]))

['adenomaWl', 'hyperplasWl', 'hyperplasNbi', 'serratedWl', 'serratedNbi', 'adenomaNbi']
class:  adenomaWl total  40
class:  hyperplasWl total  21
class:  hyperplasNbi total  21
class:  serratedWl total  15
class:  serratedNbi total  15
class:  adenomaNbi total  40


In [3]:
#Para la forma
def fd_hu_moments(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(image)).flatten()
    return feature

In [4]:
#textura
def fd_haralick(image):
    gray = cv.cvtColor(image, cv.COLOR_BAYER_BG2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [5]:
#histograma del color
def fd_histogram(image, mask= None):
    #se convierte la imagen a HSV
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    #Histograma
    hist = cv.calcHist([image], [0,1,2], None, [bins,bins,bins],[0,256,0,256,0,256])
    #normalizar hist
    cv.normalize(hist,hist)
    return hist.flatten()

In [6]:
predictor = []
target = []
resize_value = 100
for i, classes in enumerate(classes_names):
    print((i+1), "clase: ", classes, " target: ", (i+1), " total: ", len(dic_img[classes]))
    for img_name in dic_img[classes]:
        path_image = path_classes+classes+'/'+img_name
        img = cv.resize(cv.imread(path_image,0),(100,100))*1.
        flatt_img = img.reshape(resize_value*resize_value)
        ########################
        #Caracteristicas globales
        ########################
        #fv_hu_moments = fd_hu_moments(flatt_img)
        #fv_haralick = fd_haralick(flatt_img)
        #fv_histogram = fd_histogram(flatt_img)
        #######################
        #Concatenacion de todas las features
        #######################
        #predictor = np.hstack([fv_hu_moments, fv_haralick, fv_histogram])
        
        predictor.append(flatt_img)
        target.append(i+1)
predictor = np.r_[predictor]        
target = np.r_[target]

1 clase:  adenomaWl  target:  1  total:  40
2 clase:  hyperplasWl  target:  2  total:  21
3 clase:  hyperplasNbi  target:  3  total:  21
4 clase:  serratedWl  target:  4  total:  15
5 clase:  serratedNbi  target:  5  total:  15
6 clase:  adenomaNbi  target:  6  total:  40


In [4]:
print(predictor.shape)

(152, 10000)


### Normalizando 

In [17]:
predictor_norm = preprocessing.normalize(predictor)

In [18]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(predictor_norm,target,test_size=test_size)
print (y_test)

[2 2 6 3 5 4 1 6 1 1 6 1 1 2 4 6 3 1 6 2 3 5 4 2 3 1 1 1 3 1 3]


In [19]:
tree = DecisionTreeClassifier(criterion="entropy",max_depth=6, min_samples_split=20, random_state=99 )

In [20]:
tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')

In [9]:
tree.score(X_test,y_test)

0.3225806451612903

## Standardization

In [7]:
predictor_cent = preprocessing.scale(predictor)

In [8]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(predictor_cent,target,test_size=test_size)
print (y_test)

[1 6 6 2 6 3 2 6 1 1 3 6 6 6 4 6 6 4 1 2 1 3 1 6 6 6 4 3 2 3 1]


In [9]:
tree2 = DecisionTreeClassifier(criterion="entropy",max_depth=6, min_samples_split=20, random_state=99 )

In [10]:
tree2.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')

In [11]:
tree2.score(X_test,y_test)

0.41935483870967744

# <span style="color:red">Cross validation</span>

### Normalizated

In [22]:
cv = KFold(n_splits = 10, shuffle = True, random_state = 1)

In [16]:
score = np.mean(cross_val_score(tree, predictor_norm, target, scoring = "accuracy", cv = cv, n_jobs=1))
score

0.28958333333333336

### Standardization

In [17]:
cv2 = KFold(n_splits = 10, shuffle = True, random_state = 1)

In [18]:
score2 = np.mean(cross_val_score(tree2, predictor_cent, target, scoring = "accuracy", cv = cv, n_jobs=1))
score2

0.3625

# <span style="color:red">LOOCV</span>

### Normalized

In [19]:
cv3 = KFold(n_splits = predictor_norm.shape[0], shuffle = True, random_state = 1)

In [20]:
score3 = np.mean(cross_val_score(tree, predictor_norm, target, scoring = "accuracy", cv = cv, n_jobs=1))
score3

0.28958333333333336

### Standardization

In [14]:
cv4 = KFold(n_splits = predictor_cent.shape[0], shuffle = True, random_state = 1)

In [15]:
score4 = np.mean(cross_val_score(tree2, predictor_cent, target, scoring = "accuracy", cv = cv4, n_jobs=1))
score4

0.34210526315789475

# Changing deep

In [23]:
for i in range(1,11):
    tree = DecisionTreeClassifier(criterion="entropy", max_depth=i, min_samples_split=20, random_state=99 )
    tree.fit(X_train, y_train)
    score = np.mean(cross_val_score(tree, predictor_norm, target, scoring="accuracy", cv =cv, n_jobs=1))
    print("Score de:", score, "para la profundidad: ", i)
    #print("    ", tree.feature_importances_)

Score de: 0.23166666666666674 para la profundidad:  1
Score de: 0.2754166666666667 para la profundidad:  2
Score de: 0.34208333333333335 para la profundidad:  3
Score de: 0.34916666666666674 para la profundidad:  4
Score de: 0.3354166666666667 para la profundidad:  5
Score de: 0.28958333333333336 para la profundidad:  6
Score de: 0.3091666666666667 para la profundidad:  7
Score de: 0.3025 para la profundidad:  8
Score de: 0.3025 para la profundidad:  9
Score de: 0.3025 para la profundidad:  10


# <span style="color:red">DT para imagenes de varianza</span>

In [7]:
path_classes2 = "/home/franklinsierra/Documentos/Biomedica/Polipos/notebooks/mix_var_img/"
classes_names2 = os.listdir(path_classes2)
print (classes_names2)

for i, name_class in enumerate(classes_names2):
    dic_img[classes_names2[i]] = np.array(os.listdir(path_classes2+classes_names2[i]))
    print ("class: ", name_class, "total ", len(dic_img[classes_names2[i]]))

['hiperplasNbi', 'adenomaWl', 'serratedWl', 'serratedNbi', 'hiperplasWl', 'adenomaNbi']
class:  hiperplasNbi total  21
class:  adenomaWl total  40
class:  serratedWl total  15
class:  serratedNbi total  15
class:  hiperplasWl total  21
class:  adenomaNbi total  40


In [8]:
predictor2 = []
target2 = []
resize_value = 100
for i, classes in enumerate(classes_names2):
    print((i+1), "clase: ", classes, " target: ", (i+1), " total: ", len(dic_img[classes]))
    for img_name in dic_img[classes]:
        path_image = path_classes2+classes+'/'+img_name
        img = cv.resize(cv.imread(path_image,0),(100,100))*1.
        flatt_img = img.reshape(resize_value*resize_value)
        ########################
        #Caracteristicas globales
        ########################
        #fv_hu_moments = fd_hu_moments(flatt_img)
        #fv_haralick = fd_haralick(flatt_img)
        #fv_histogram = fd_histogram(flatt_img)
        #######################
        #Concatenacion de todas las features
        #######################
        #predictor = np.hstack([fv_hu_moments, fv_haralick, fv_histogram])
        
        predictor2.append(flatt_img)
        target2.append(i+1)
predictor2 = np.r_[predictor2]        
target2 = np.r_[target2]

1 clase:  hiperplasNbi  target:  1  total:  21
2 clase:  adenomaWl  target:  2  total:  40
3 clase:  serratedWl  target:  3  total:  15
4 clase:  serratedNbi  target:  4  total:  15
5 clase:  hiperplasWl  target:  5  total:  21
6 clase:  adenomaNbi  target:  6  total:  40


### Normalizando

In [9]:
predictor2_norm = preprocessing.normalize(predictor2)

In [10]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train2, X_test2, y_train2, y_test2 = train_test_split(predictor2_norm,target2,test_size=test_size)
print (y_test2)

[2 5 5 5 6 1 1 2 5 5 2 2 2 6 4 6 4 6 5 6 2 2 4 6 6 4 2 2 6 3 6]


In [11]:
tree3 = DecisionTreeClassifier(criterion="entropy",max_depth=6, min_samples_split=20, random_state=99 )

In [12]:
tree3.fit(X_train2,y_train2)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')

In [13]:
tree3.score(X_test2, y_test2)

0.22580645161290322

### Standardization

In [14]:
predictor2_cent = preprocessing.scale(predictor2)

In [16]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X2_train, X2_test, y2_train, y2_test = train_test_split(predictor2_cent,target2,test_size=test_size)
print (y2_test)

[2 6 6 4 1 2 5 2 6 2 6 6 5 3 6 2 6 1 2 2 3 5 6 2 2 2 5 6 2 6 6]


In [17]:
tree4 = DecisionTreeClassifier(criterion="entropy",max_depth=6, min_samples_split=20, random_state=99 )

In [18]:
tree4.fit(X_train2,y_train2)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')

In [19]:
tree4.score(X2_test, y2_test)

0.25806451612903225

## LOOCV

### Normalizated

In [23]:
valid = KFold(n_splits = predictor2_norm.shape[0], shuffle = True, random_state = 1)

In [24]:
acc = np.mean(cross_val_score(tree3, predictor2_norm, target2, scoring = "accuracy", cv = valid, n_jobs=1))
acc

0.23684210526315788

### Standardization

In [27]:
valid = KFold(n_splits = predictor2_cent.shape[0], shuffle = True, random_state = 1)

In [45]:
acc = np.mean(cross_val_score(tree4, predictor2_cent, target2, scoring = "accuracy", cv = valid, n_jobs=1))
acc

0.3618421052631579

## Changing deep

In [28]:
for i in range(1,11):
    tree2 = DecisionTreeClassifier(criterion="entropy", max_depth=i, min_samples_split=20, random_state=99 )
    tree2.fit(X_train2, y_train2)
    score = np.mean(cross_val_score(tree2, predictor2_norm, target2, scoring="accuracy", cv =valid, n_jobs=1))
    print("Score de:", score, "para la profundidad: ", i)

Score de: 0.34868421052631576 para la profundidad:  1


KeyboardInterrupt: 

# <span style="color:red">DT para las derivadas de las imagenes </span>

In [29]:
path_classes3 = "/home/franklinsierra/Documentos/Biomedica/Polipos/notebooks/mix_der_img/"
classes_names3 = os.listdir(path_classes3)
print (classes_names3)

for i, name_class in enumerate(classes_names3):
    dic_img[classes_names3[i]] = np.array(os.listdir(path_classes3+classes_names3[i]))
    print ("class: ", name_class, "total ", len(dic_img[classes_names3[i]]))

['hiperplasNbi', 'adenomaWl', 'serratedWl', 'serratedNbi', 'hiperplasWl', 'adenomaNbi']
class:  hiperplasNbi total  21
class:  adenomaWl total  40
class:  serratedWl total  15
class:  serratedNbi total  15
class:  hiperplasWl total  21
class:  adenomaNbi total  40


In [30]:
predictor3 = []
target3 = []
resize_value = 100
for i, classes in enumerate(classes_names3):
    print((i+1), "clase: ", classes, " target: ", (i+1), " total: ", len(dic_img[classes]))
    for img_name in dic_img[classes]:
        path_image = path_classes3+classes+'/'+img_name
        img = cv.resize(cv.imread(path_image,0),(100,100))*1.
        flatt_img = img.reshape(resize_value*resize_value)
        ########################
        #Caracteristicas globales
        ########################
        #fv_hu_moments = fd_hu_moments(flatt_img)
        #fv_haralick = fd_haralick(flatt_img)
        #fv_histogram = fd_histogram(flatt_img)
        #######################
        #Concatenacion de todas las features
        #######################
        #predictor = np.hstack([fv_hu_moments, fv_haralick, fv_histogram])
        
        predictor3.append(flatt_img)
        target3.append(i+1)
predictor3 = np.r_[predictor3]        
target3 = np.r_[target3]

1 clase:  hiperplasNbi  target:  1  total:  21
2 clase:  adenomaWl  target:  2  total:  40
3 clase:  serratedWl  target:  3  total:  15
4 clase:  serratedNbi  target:  4  total:  15
5 clase:  hiperplasWl  target:  5  total:  21
6 clase:  adenomaNbi  target:  6  total:  40


### Normalizando datos

In [31]:
predictor3_norm = preprocessing.normalize(predictor3)

In [32]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train3, X_test3, y_train3, y_test3 = train_test_split(predictor3_norm,target3,test_size=test_size)
print (y_test3)

[2 2 6 3 2 2 2 6 1 6 2 2 3 1 3 1 2 1 3 6 2 2 4 2 6 4 2 6 3 5 4]


In [33]:
tree5 = DecisionTreeClassifier(criterion="entropy",max_depth=6, min_samples_split=20, random_state=99 )

In [34]:
tree5.fit(X_train3,y_train3)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')

In [35]:
tree5.score(X_test3, y_test3)

0.25806451612903225

## Standardization

In [36]:
predictor3_cent = preprocessing.scale(predictor3)

In [37]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train3, X_test3, y_train3, y_test3 = train_test_split(predictor3_cent,target3,test_size=test_size)
print (y_test3)

[2 5 5 5 2 5 2 1 6 5 5 3 2 5 6 1 6 6 3 1 5 1 4 1 6 6 6 3 2 2 6]


In [38]:
tree6 = DecisionTreeClassifier(criterion="entropy",max_depth=6, min_samples_split=20, random_state=99 )

In [39]:
tree6.fit(X_train3,y_train3)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')

In [40]:
tree6.score(X_test3, y_test3)

0.2903225806451613

## <span style="color:red">LOOCV</span>

## Normalized

In [41]:
valid6 = KFold(n_splits = predictor3_norm.shape[0], shuffle = True, random_state = 1)

In [42]:
score6 = np.mean(cross_val_score(tree5, predictor3_norm, target3, scoring = "accuracy", cv = valid6, n_jobs=1))
score6

0.34210526315789475

## Standardization

In [43]:
valid7 = KFold(n_splits = predictor3_cent.shape[0], shuffle = True, random_state = 1)

In [44]:
score7 = np.mean(cross_val_score(tree6, predictor3_cent, target3, scoring = "accuracy", cv = valid7, n_jobs=1))
score7

0.18421052631578946

## Changing deep

In [ ]:
for i in range(1,11):
    tree3 = DecisionTreeClassifier(criterion="entropy", max_depth=i, min_samples_split=20, random_state=99 )
    tree3.fit(X_train3, y_train3)
    score = np.mean(cross_val_score(tree3, predictor3_norm, target3, scoring="accuracy", cv =cv2, n_jobs=1))
    print("Score de:", score, "para la profundidad: ", i)